# Projet

## Collecte de données

In [25]:
!pip install sparqlwrapper
!pip install pandas


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# Get casino image
query = """SELECT ?image {
  ?data wdt:P31 wd:Q133215;
        wdt:P18 ?image
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["image"]["value"],
        )
    )

In [27]:
import pandas as pd
dataframe = pd.DataFrame(array, columns=["image"])
dataframe = dataframe.astype(
    dtype={"image": "<U200"}
)
dataframe

,image
0,http://commons.wikimedia.org/wiki/Special:File...
1,http://commons.wikimedia.org/wiki/Special:File...
2,http://commons.wikimedia.org/wiki/Special:File...
3,http://commons.wikimedia.org/wiki/Special:File...
4,http://commons.wikimedia.org/wiki/Special:File...
...,...
418,http://commons.wikimedia.org/wiki/Special:File...
419,http://commons.wikimedia.org/wiki/Special:File...
420,http://commons.wikimedia.org/wiki/Special:File...
421,http://commons.wikimedia.org/wiki/Special:File...


In [28]:
import os
try:
    os.mkdir('images')
except:
    pass

Nous allons écrire une méthode pour extraire les exifs

Nous allons maintenant télécharger les images

In [29]:
import requests
import shutil
import os
import json

def download_image(url):
    global metadata
    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        nom = os.path.basename(url)[-30:]
        chemin = "images/"+nom
        with open(chemin, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

In [30]:
dataframe.image.apply(download_image)

0      200
1      200
2      200
3      200
4      200
      ... 
418    200
419    200
420    200
421    200
422    200
Name: image, Length: 423, dtype: int64

On enregistre les exifs

In [ ]:
import os, sys, json
from PIL import Image
from PIL.ExifTags import TAGS
import numpy
import math
from sklearn.cluster import KMeans

def extraire_couleurs(image_chemin):
    imgfile = Image.open(image_chemin)
    numarray = numpy.array(imgfile.getdata(), numpy.uint8)
    print(image_chemin)
    clusters = KMeans(n_clusters=5, n_init=2)
    couleurs = []
    try :
        clusters.fit(numarray)
        for i in range(5):
            couleur = ("#%02x%02x%02x"
                % (
                    math.ceil(clusters.cluster_centers_[i][0]),
                    math.ceil(clusters.cluster_centers_[i][1]),
                    math.ceil(clusters.cluster_centers_[i][2]),
                ))
            couleurs.append(couleur)
    except:
        print(f"les couleurs n'ont pas pu être extraits depuis {image_chemin}")
        couleur = "#00000"
        for i in range(5):
            couleurs.append(couleur)
    print(couleurs)
    return couleurs


def extraire_donnees(image_chemin):
    imgfile = Image.open(image_chemin)
    try:
        exif_data = imgfile._getexif()
    except:
        print(f"les EXIFs n'ont pas pu être extraits depuis {image_chemin}")
        exif_data = {}
    image_metadata = dict()
    if exif_data:
        for tag, value in exif_data.items():
            if(isinstance(value, bytes)):
                continue
            if (not isinstance(value, int)) and (not isinstance(value, float)):
                value = f"{value}"
            if tag in TAGS:
                image_metadata[TAGS[tag]] = value
    print(image_metadata)
    return image_metadata

json_dat = {}
for filename in os.listdir("images"):
    json_dat[filename] = extraire_donnees("images/"+filename)
    extraire_couleurs("images/"+filename)
json.dump(json_dat, open("img_data.json", "w"), indent=2)

{'ResolutionUnit': 2, 'ExifOffset': 2356, 'ProcessingSoftware': 'Windows Photo Editor 10.0.10011.16384', 'Make': 'Canon', 'Model': 'Canon MG3600 series', 'Software': 'Windows Photo Editor 10.0.10011.16384', 'Orientation': 1, 'DateTime': '2019:01:11 19:23:54', 'YCbCrPositioning': 1, 'XResolution': '600.0', 'YResolution': '600.0', 'DateTimeOriginal': '2019:01:11 19:21:44', 'DateTimeDigitized': '2019:01:11 19:21:44', 'ColorSpace': 1, 'ExifImageWidth': 4872, 'ExifImageHeight': 5304, 'SubsecTimeOriginal': '32', 'SubsecTimeDigitized': '32'}
images/%20%E2%80%94%20Le%20Casino.jpg
['#585357', '#b5adab', '#878183', '#312c31', '#d7cec6']
{'ResolutionUnit': 2, 'ExifOffset': 256, 'ImageDescription': '                               ', 'Make': 'SONY', 'Model': 'DSC-HX1', 'Orientation': 1, 'DateTime': '2012:04:11 15:31:07', 'YCbCrPositioning': 2, 'XResolution': '72.0', 'YResolution': '72.0', 'CompressedBitsPerPixel': '5.0', 'DateTimeOriginal': '2012:04:11 15:31:07', 'DateTimeDigitized': '2012:04:11 15